# Deep Nets with TF Abstractions

Let's explore a few of the various abstractions that TensorFlow offers. You can check out the tf.contrib documentation for more options.

# The Data

To compare these various abstractions we'll use a dataset easily available from the SciKit Learn library. The data is comprised of the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators. There are thirteen different
measurements taken for different constituents found in the three types of wine. We will use the various TF Abstractions to classify the wine to one of the 3 possible labels.

First let's show you how to get the data:

In [1]:
import sklearn.datasets

In [2]:
wine_data = sklearn.datasets.load_wine()

In [3]:
type(wine_data)

sklearn.utils.Bunch

In [4]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [5]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [7]:
feat_data = wine_data['data']
labels = wine_data['target']

### Train Test Split

In [8]:
import sklearn.model_selection

In [9]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(feat_data,
                                                                            labels,
                                                                            test_size = 0.3,
                                                                            random_state = 101)

### Scale the Data

In [14]:
import sklearn.preprocessing

In [15]:
scaler = sklearn.preprocessing.MinMaxScaler()
scaled_x_train = scaler.fit_transform(x_train)
scaled_x_test = scaler.transform(x_test)

# Abstractions

## Estimator API

In [16]:
import tensorflow as tf

/Users/johnleonard/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [20]:
x_train_shape = x_train.shape
x_train_shape

(124, 13)

In [19]:
feat_cols = [tf.feature_column.numeric_column('x',shape = [13])]

In [22]:
deep_model = tf.estimator.DNNClassifier(hidden_units = [13,13,13], 
                                        feature_columns = feat_cols,
                                        n_classes = 3,
                                        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/dk/jmp2jdj96xd00z6w7mbmgvh40000gn/T/tmp_7avpzx1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a261cb7f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
input_fn = tf.estimator.inputs.numpy_input_fn(x = {'x':scaled_x_train},
                                              y = y_train,
                                              shuffle = True,
                                              batch_size = 10,
                                              num_epochs = 5)

In [25]:
deep_model.train(input_fn=input_fn, steps = 500)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/dk/jmp2jdj96xd00z6w7mbmgvh40000gn/T/tmp_7avpzx1/model.ckpt.
INFO:tensorflow:loss = 11.2792225, step = 1
INFO:tensorflow:Saving checkpoints for 62 into /var/folders/dk/jmp2jdj96xd00z6w7mbmgvh40000gn/T/tmp_7avpzx1/model.ckpt.
INFO:tensorflow:Loss for final step: 6.2916193.


In [26]:
input_fn_eval = tf.estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [29]:
preds_dict = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/dk/jmp2jdj96xd00z6w7mbmgvh40000gn/T/tmp_7avpzx1/model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [30]:
preds_dict

[{'logits': array([ 1.3203056 ,  0.66884035, -0.47925538], dtype=float32),
  'probabilities': array([0.5928902 , 0.30906266, 0.09804712], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([ 1.6883984 ,  0.72956645, -0.62155837], dtype=float32),
  'probabilities': array([0.67448807, 0.25855854, 0.06695343], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-0.59280056,  0.45276704,  0.9024872 ], dtype=float32),
  'probabilities': array([0.12040021, 0.34254014, 0.5370596 ], dtype=float32),
  'class_ids': array([2]),
  'classes': array([b'2'], dtype=object)},
 {'logits': array([ 2.1241436 ,  0.73775977, -0.7794189 ], dtype=float32),
  'probabilities': array([0.76639795, 0.19158235, 0.04201972], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-0.5677726 ,  0.45036754,  0.9193898 ], dtype=float32),
  'probabilities': array([0.1

In [31]:
preds = [p['class_ids'][0] for p in preds]

In [33]:
import sklearn.metrics

In [34]:
print(sklearn.metrics.classification_report(y_test,preds))

             precision    recall  f1-score   support

          0       0.70      1.00      0.83        19
          1       1.00      0.59      0.74        22
          2       0.93      1.00      0.96        13

avg / total       0.88      0.83      0.83        54



____________
______________

# TensorFlow Keras

In [35]:
import tensorflow as tf

In [36]:
from tensorflow.contrib import keras

### Create the Model

In [38]:
dnn_model = keras.models.Sequential()

### Add Layers to the model

In [40]:
dnn_model.add(keras.layers.Dense(units=13, input_dim = 13, activation = 'relu'))
dnn_model.add(keras.layers.Dense(units=13, input_dim = 13, activation = 'relu'))
dnn_model.add(keras.layers.Dense(units=13, input_dim = 13, activation = 'relu'))
dnn_model.add(keras.layers.Dense(units = 3, activation='softmax'))

### Compile the Model

In [45]:
dnn_model.compile(optimizer='adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy']) # Use sparse on categories that aren't one-hot encoded. If you one-hot encode, just use categorical crossentropy

### Train Model

In [46]:
dnn_model.fit(scaled_x_train, y_train, epochs=50)

Epoch 1/50
124/124 [==============================] - 1s 4ms/step - loss: 0.1945 - acc: 0.9919
Epoch 2/50
124/124 [==============================] - 0s 90us/step - loss: 0.1831 - acc: 0.9839
Epoch 3/50
124/124 [==============================] - 0s 106us/step - loss: 0.1709 - acc: 0.9919
Epoch 4/50
124/124 [==============================] - 0s 112us/step - loss: 0.1633 - acc: 0.9919
Epoch 5/50
124/124 [==============================] - 0s 118us/step - loss: 0.1512 - acc: 0.9919
Epoch 6/50
124/124 [==============================] - 0s 148us/step - loss: 0.1459 - acc: 0.9919
Epoch 7/50
124/124 [==============================] - 0s 138us/step - loss: 0.1364 - acc: 0.9919
Epoch 8/50
124/124 [==============================] - 0s 115us/step - loss: 0.1317 - acc: 0.9919
Epoch 9/50
124/124 [==============================] - 0s 104us/step - loss: 0.1232 - acc: 0.9919
Epoch 10/50
124/124 [==============================] - 0s 111us/step - loss: 0.1161 - acc: 0.9919
Epoch 11/50
124/124 [===========

In [47]:
preds = dnn_model.predict_classes(scaled_x_test)

In [49]:
print(sklearn.metrics.classification_report(y_test,preds))

             precision    recall  f1-score   support

          0       0.86      1.00      0.93        19
          1       1.00      0.86      0.93        22
          2       1.00      1.00      1.00        13

avg / total       0.95      0.94      0.94        54



____________
______________

# Layers API

https://www.tensorflow.org/tutorials/layers

In [51]:
import tensorflow as tf
import pandas as pd

## Formating Data

In [58]:
onehot_y_train = pd.get_dummies(y_train).values
onehot_y_test = pd.get_dummies(y_test).values

### Parameters

In [59]:
n_features = 13
n_HL1 = 13
n_HL2 = 13
n_outputs = 3

learning_rate = 0.01

### Placeholder

In [60]:
x = tf.placeholder(tf.float32, shape = [None, n_features])
y_true = tf.placeholder(tf.float32, shape = [None, n_outputs])

### Activation Function

In [61]:
act_fxn = tf.nn.relu

### Create Layers

In [63]:
HL1 = tf.contrib.layers.fully_connected(x, n_HL1, activation_fn= act_fxn)
HL2 = tf.contrib.layers.fully_connected(HL1, n_HL2, activation_fn=act_fxn)

output = tf.contrib.layers.fully_connected(HL2, n_outputs)

### Loss Function

In [64]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits= output)

### Optimizer

In [67]:
optimizer = tf.train.AdamOptimizer(learning_rate)
minimizer = optimizer.minimize(loss)

### Init

In [77]:
init = tf.global_variables_initializer()
training_steps = 10000

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(minimizer, feed_dict = {x: scaled_x_train, y_true: onehot_y_train})
        
    logits = output.eval(feed_dict={x:scaled_x_test})
    preds = tf.argmax(logits, axis=1)
    results = preds.eval()
    

In [78]:
print(sklearn.metrics.classification_report(y_test,results))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       1.00      1.00      1.00        22
          2       1.00      1.00      1.00        13

avg / total       1.00      1.00      1.00        54

